In [ ]:
# !pip install ogame

In [ ]:
# https://github.com/alaingilbert/pyogame/wiki/Code-Snippets

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../pyogame'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from ogame import OGame, constants
import time

In [3]:
o = OGame(
    universe="Ymir",
#     universe='Volans',
    username="samchakerian@gmail.com",
    password="$R3e@W1q1q"
)

In [ ]:
# # time 
# sp = 5
# mm = 3
# cm = 3
# dm = 15

In [ ]:
# income
# mm1 : +504
# mm2: ???
# mm3: +969
# cm : +120
# cm2: +414

In [ ]:
# make class with starting res, build times, income

# should initiate building, unless you need to wait for resources

# in that case, should calculate wait time



In [4]:
# id = o.planet_ids()[0]
# o.build(constants.buildings.solar_plant, id)
# time.sleep(sp)
# o.build(constants.buildings.metal_mine, id)
# time.sleep(mm)
# o.build(constants.buildings.metal_mine, id)
# time.sleep(mm*2)
# o.build(constants.buildings.solar_plant, id)
# time.sleep(sp*2)
# o.build(constants.buildings.metal_mine, id)
# time.sleep(mm*3)
# o.build(constants.buildings.metal_mine, id)
# time.sleep(mm*4)
# o.build(constants.buildings.solar_plant, id)
# time.sleep(sp*3)
# o.build(constants.buildings.crystal_mine, id)
# time.sleep(cm)
# o.build(constants.buildings.solar_plant, id)
# time.sleep(sp*4)
# o.build(constants.buildings.metal_mine, id)
# time.sleep(mm*5)
# o.build(constants.buildings.crystal_mine, id)
# time.sleep(cm*2)
# o.build(constants.buildings.crystal_mine, id)
# time.sleep(cm*3)
# o.build(constants.buildings.solar_plant, id)
# time.sleep(sp*5)
# o.build(constants.buildings.deuterium_mine, id)
# time.sleep(dm)

In [ ]:
# # https://ogame.fandom.com/wiki/Quick_Start_Guide

# Step // Building/Tech // Building Tech Level // notes // (Sam time)

# 1 // Solar Plant // 1 // 5s
# 2 // Metal mine // 1 // 3s
# 3 // Metal Mine // 2 // 
# 4 // Solar Plant // 2 // 
# 5 // Metal Mine // 3 // +8 Energy
# 6 // Metal Mine // 4 // - 11 Energy but improvement in production
# 7 // Solar Plant // 3
# 8 // Crystal Mine // 1 // +9 energy. Do not upgrade. Stick to the guide
# 9 // Solar Plant // 4
# 10 // Metal Mine // 5 // You can also switch steps 10 and 11 if you're doing the tutorial to get free rss
# 11 // Crystal Mine // 2 // +11 energy
# 12 // Crystal Mine // 3 // -4 energy but do upgrade because improvement in production. 
# Metal prod decreases with 9 metal/h no biggy
# 13 // Solar Plant // 5 // Make sure to build this as soon as you can to reduce unnecessary production loss

# 14 // Deuterium Synthesizer // 1 // Energy +18

# 15 // Crystal Mine // 4 // Energy -1 Upgrade in production so build it.

# 16 // Solar Plant // 6

# 17 // Metal Mine // 6

# 18 // Metal Mine // 7 // -6 but improvement (produce at 90%)

# 19 // Solar Plant // 7 // put Metal Mine back to 100% after done

# 20 // Crystal Mine // 5

# 21 // Deuterium Synthesizer // 2

# 22 // Solar Plant // 8

# 23 // Deuterium Synthesizer // 3

# 24 // Deuterium Synthesizer // 4

# 25 // Solar Plant // 9

# 26 // Deuterium Synthesizer // 5

# 27 // Robotics Factory // 1 // Robo 1 is the first building that costs deuterium. 
# Continue with the list in the given order as much as you can. If you dont have deuterium left, 
# then just continue with non deuterium costing buildings.

# 28 // Robotics Factory // 2

# 29 // Research Lab // 1

# 30 // Shipyard // 1

# 31 // Crystal Mine // 6

# 32 // Shipyard // 2

# 33 // Solar Plant // 10

# 34 // Deuterium Synthesizer // 6

# 35 // Metal Mine // 8

# 36 // Energy Research // 1

# 37 // Combustion Drive // 1 // If you have enough resources

# 38 // Solar Plant // 11 // Whenever you can research combustion drive 2, research it.

# 39 // Crystal Mine // 7 // Whenever you can build Shipyard 3, build it

# 40 // Metal Mine // 9



In [5]:
id = o.planet_ids()[0]
supply = o.supply(id)
res = o.resources(id)

In [6]:
# check bldg construction
{
    'sp': supply.solar_plant.in_construction,
    'mm': supply.metal_mine.in_construction,
    'cm': supply.crystal_mine.in_construction,
    'dm': supply.deuterium_mine.in_construction
}

{'sp': False, 'mm': False, 'cm': False, 'dm': False}

In [7]:
# is possible
{
    'sp': supply.solar_plant.is_possible,
    'mm': supply.metal_mine.is_possible,
    'cm': supply.crystal_mine.is_possible,
    'dm': supply.deuterium_mine.is_possible
}

{'sp': True, 'mm': True, 'cm': True, 'dm': True}

In [8]:
# check resources
{
    'sp': res.energy,
    'mm': res.metal,
    'cm': res.crystal,
    'dm': res.deuterium
}

{'sp': 418, 'mm': 53440, 'cm': 40000, 'dm': 33154}

In [9]:
# check levels
{
    'sp': supply.solar_plant.level,
    'mm': supply.metal_mine.level,
    'cm': supply.crystal_mine.level,
    'dm': supply.deuterium_mine.level
}

{'sp': 14, 'mm': 10, 'cm': 8, 'dm': 8}

In [ ]:
# o.is_logged_in()

# o.relogin()

# o.logout()

In [ ]:
# # keep_going decorator
# @empire.keep_going
# def run():
#     while True:
#         print(empire.attacked())
#         time.sleep(1)

In [ ]:
# attack inactives

# # Send a Spyprobe to a random Player
# send_fleet = False
# while not send_fleet:
#     for planet in empire.galaxy(coordinates(randint(1,6), randint(1, 499))):
#         if empire.send_fleet(mission.spy, id, planet.position, [ships.espionage_probe(1)]):
#             send_fleet = True
#             break

# # Wait till Spyprobe arrives
# for fleet in empire.fleet():
#     if fleet.mission == mission.spy:
#         while datetime.now() < fleet.arrival:
#             continue

# # Print out Spyreport
# for spyreport in o.spyreports():
#     print(spyreport.list)

# for planet in empire.galaxy(coordinates(1, 1)):
#     if status.inactive in planet.status and status.vacation not in planet.status:
#         empire.send_fleet(mission=mission.attack,
#                           id=empire.id_by_planet_name('MAIN_PLANET'),
#                           where=coordinates(1, 1, planet.position),
#                           ships=[ships.large_transporter(25), ships.cruiser(1)])

In [ ]:
o.planet_ids()[0]

In [ ]:
o.celestial_coordinates(33624031)

In [34]:
# for planet in o.galaxy(constants.coordinates(1, 447)):
#     print(planet.status)
    
for galaxy in range(441, 455):
    for planet in o.galaxy(constants.coordinates(1, galaxy)):
        # print(galaxy, planet.status, planet.name, planet.rank)
        if 'inactive' in planet.status and 'vacation' not in planet.status:
            print(galaxy, planet.status, planet.name, planet.rank)
            o.send_fleet(
                constants.mission.spy, 
                o.planet_ids()[0], 
                planet.position, 
                [constants.ships.espionage_probe(1)]
            )

441 ['inactive'] Homeworld 1453
444 ['inactive'] Homeworld 1431
444 ['inactive'] Howak 1116
445 ['inactive'] Homeworld 1345
445 ['inactive'] Homeworld 1132
448 ['inactive'] Homeworld 1570
449 ['inactive'] Homeworld 1627
450 ['inactive'] Beer 560
451 ['inactive'] Homeworld 1522
451 ['inactive'] Homeworld 1523
452 ['inactive'] Homeworld 1255
453 ['inactive'] Homeworld 1103


In [21]:
response = o.session.get(o.index_php + 'page=ingame&component=fleetdispatch&cp={}'.format(33624031))

In [35]:
# response.text

In [13]:
import re

In [23]:
send_fleet_token = re.search(
    # 'var fleetSendingToken = "(.*)"',
    'var token = "(.*)"',
    response.text
).group(1)
form_data = {'token': send_fleet_token}

In [24]:
form_data

{'token': 'b13bf8f7e593865de07d7e057f8b8e3c'}

In [25]:
ships = [constants.ships.espionage_probe(1)]

In [26]:
send_fleet_token = 'b13bf8f7e593865de07d7e057f8b8e3c' # from inspect

where = [1, 440, 8, 1]
resources = (0, 0, 0)
mission = constants.mission.spy
speed = 10
holdingtime = 0

form_data = {'token': send_fleet_token}

for ship in ships:
    ship_type = 'am{}'.format(ship[0])
    form_data.update({ship_type: ship[1]})

form_data.update(
    {
        'galaxy': where[0],
        'system': where[1],
        'position': where[2],
        'type': where[3],
        'metal': resources[0],
        'crystal': resources[1],
        'deuterium': resources[2],
        'prioMetal': 1,
        'prioCrystal': 2,
        'prioDeuterium': 3,
        'mission': mission,
        'speed': speed,
        'retreatAfterDefenderRetreat': 0,
        'union': 0,
        'holdingtime': holdingtime
    }
)

# response = self.session.post(
#     url=self.index_php + 'page=ingame&component=fleetdispatch'
#         '&action=sendFleet&ajax=1&asJson=1',
#     data=form_data,
#     headers={'X-Requested-With': 'XMLHttpRequest'}
# ).json()
# return response['success']

In [27]:
form_data

{'token': 'b13bf8f7e593865de07d7e057f8b8e3c',
 'am210': 1,
 'galaxy': 1,
 'system': 440,
 'position': 8,
 'type': 1,
 'metal': 0,
 'crystal': 0,
 'deuterium': 0,
 'prioMetal': 1,
 'prioCrystal': 2,
 'prioDeuterium': 3,
 'mission': 6,
 'speed': 10,
 'retreatAfterDefenderRetreat': 0,
 'union': 0,
 'holdingtime': 0}

In [28]:
response = o.session.post(
    url=o.index_php + 'page=ingame&component=fleetdispatch&action=sendFleet&ajax=1&asJson=1',
    data=form_data,
    headers={'X-Requested-With': 'XMLHttpRequest'}
)

In [29]:
response.json()

{'success': True,
 'message': 'Your fleet has been successfully sent.',
 'token': 'eecf50d9b80a35931be4e609e138251e',
 'redirectUrl': 'https://s177-en.ogame.gameforge.com/game/index.php?page=ingame&component=fleetdispatch',
 'components': [],
 'newAjaxToken': 'eecf50d9b80a35931be4e609e138251e'}

In [30]:
# id = o.planet_ids()[0]

In [31]:
# from ogame.constants import coordinates, mission, speed, fleet

In [32]:
# fleet

In [33]:
# # from ogame.constants import coordinates, mission, speed, fleet
# o.send_fleet(mission=mission.expedition,
#                   id=id,
#                   where=coordinates(1, 440, 8),
#                   ships=fleet(espionage_probe=1),
#                   resources=[0, 0, 0],  # optional default no resources
#                   speed=speed.max,      # optional default speed.max
#                   holdingtime=2)        # optional default 0 will be needed by expeditions

In [36]:
# o.spyreports()
o.spyreports()[0].fright

[('defense401', '4'),
 ('building1', '5'),
 ('building22', '2'),
 ('building2', '5'),
 ('building23', '2'),
 ('building3', '5'),
 ('building24', '1'),
 ('building4', '7'),
 ('building14', '3'),
 ('building21', '1'),
 ('building31', '3'),
 ('research113', '1'),
 ('research115', '1')]

In [41]:
from bs4 import BeautifulSoup

def BeautifulSoup4(response):
    parsed = BeautifulSoup(response, features="html5lib")

    def find_partial(**kwargs):
        for key, value in kwargs.items():
            kwargs[key] = re.compile(value)
        return parsed.find(**kwargs)

    def find_all_partial(**kwargs):
        for key, value in kwargs.items():
            kwargs[key] = re.compile(value)
        return parsed.find_all(**kwargs)

    parsed.find_partial = find_partial
    parsed.find_all_partial = find_all_partial
    return parsed

In [40]:
o.planet_ids()

[33624031]

In [42]:
response = o.session.get(
    url=o.index_php,
    params={'page': 'messages',
            'tab': 20,
            'ajax': 1}
).text
bs4 = BeautifulSoup4(response)
report_links = [
    link['href']
    for link in bs4.find_all_partial(href='page=messages&messageId')
]

In [43]:
report_links

['https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989723&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989715&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989712&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989705&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989698&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989696&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989694&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989686&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=989680&tabid=20&ajax=1',
 'https://s177-en.ogame.gameforge.com/game/index.php?page=messages&messageId=98967

In [44]:
data = []

reports = []
for link in report_links:
    response = o.session.get(link).text
    bs4 = BeautifulSoup4(response)
#     technologys = [tech['class'][0] for tech in bs4.find_all('img')]
#     amounts = [
#         tech.parent.parent.find_all('span')[1].text
#         for tech in bs4.find_all('img')
#     ]
    data.append({'link': link, 'text': bs4})

ConnectionError: HTTPSConnectionPool(host='s177-en.ogame.gameforge.com', port=443): Max retries exceeded with url: /game/index.php?page=messages&messageId=989712&tabid=20&ajax=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000023E300729C8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [46]:
len(data)

2

In [70]:
# data[0]['text'].find_all('span', class_="res_value")

# data[0]['text'].find('span', class_="msg_title new blue_txt").text

[_.text for _ in data[0]['text'].find('ul', attrs={"data-type": "defense"}).find('li').find_all('span')]

['Rocket Launcher', '4']